In [8]:
# Customer Personality Analysis

In [14]:
import pandas as pd
import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline  #设置图像显示方式
sns.set()    #设置画图空间为 Seaborn 默认风格

# plt.rcParams['axes.unicode_minus'] = False # 解决负号'-'显示为方块的问题


UsageError: unrecognized arguments: #设置图像显示方式


In [15]:
names=['CRIM','ZN','INDUS','CHAS','NOX','RM','GE','DIS','RAD','TAX','PRTATIO','B','LSTAT','PRICE']
df=pd.read_csv('housing.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: 'housing.csv'

In [ ]:
#指标解释
#CRIM犯罪率
#ZN住宅用地所占比例
#INDUS城镇中非住宅用地所占比例
#CHAS是否穿过查尔斯河
#NOX氮氧化污染物
#RM每栋住宅的房间数
#GE1940年以前建成的自住单位的比例
#DIS距离5个波士顿的就业中心的加权距离
#RAD距离高速公路的便利指数
#TAX每一万美元的不动产税率
#PRTATIO城镇中的教师学生比例
#B城镇中的黑人比例
#LSTAT低收入群比例
#PRICE自住房屋房价中位数（也就是均价）

一、数据预处理

In [ ]:
#查看数据集大小，(行,列)
df.shape

In [ ]:
# 查看各字段基础信息
df.info()

In [ ]:
#查看缺失值
df.isnull().sum()

二、描述性数据分析

①查看描述性数据统计：可以看到各个字段的均值、中位数、标准差等。

In [ ]:
#描述性数据统计
df.describe()

In [ ]:
#方差  单列数据
df['PRICE'].var()

In [ ]:
# 所有数据的方差
df.var()

In [ ]:
#标准差
df['PRICE'].std()

In [ ]:
# 我们可以画图查看分布
# 房屋总价分布
import matplotlib.pyplot as plt
plt.hist(df['PRICE'], bins=50, color='steelblue', normed=True ) #normed 计算每个区间的概率密度

In [ ]:
# a=plt.hist(df['PRICE'], bins=50, color='steelblue', normed=True )
# print(a) #a[0] 每个区间的概率，a[1] 区间划分

In [ ]:
#`RM`: 住宅平均房间数量的直方分布
plt.hist(df['RM'], bins=50, color='steelblue' )

②查看是否穿过查尔斯河对房价的影响

In [ ]:
#查看是否被查尔斯河穿过的房子的占比
print(df['CHAS'].value_counts())

fig,ax = plt.subplots(1,2,figsize=(10,5))

# 用pandas画图
df['CHAS'].value_counts().plot.pie(ax=ax[0],shadow=False,autopct='%1.2f%%')
ax[0].set_ylabel('')    #设置y轴标签
ax[0].set_xlabel('CHAS')    #设置x轴标签

# 用seaborn画图
sns.countplot('CHAS',data=df,ax=ax[1])
ax[1].set_ylabel('')
ax[1].set_xlabel('CHAS')
plt.show()

可以看到被河流穿过的豪宅仅占比6.92%

In [ ]:
#再来看看两种不同类型的房子的价值如何

bostonCHAS = df[['CHAS','PRICE']]    #先将CHAS和PRICE两列数据取出

bostonCHAS1=bostonCHAS.pivot_table(values='PRICE',    #计算的值
                               index='CHAS',       #透视的行，分组的依据
                               aggfunc='mean')            #聚合函数

# 对透视表进行降序排列
bostonCHAS1 = bostonCHAS1.sort_values(by='PRICE',     # 排序依据
                        ascending=False                 # 是否升序排列
                       )

bostonCHAS1

In [ ]:
print(bostonCHAS1.iloc[0,0])
improve=(bostonCHAS1.iloc[0,0]-bostonCHAS1.iloc[1,0])/bostonCHAS1.iloc[1,0]
print("价格提升：%s" % improve)

可以看到被查尔斯河穿过的豪宅，比没有被穿过的豪宅平均贵了28.7%

In [ ]:
# 绘制相关性曲线：RM每栋住宅的房间数与PRICE房屋价格
plt.rcParams['font.sans-serif'] = ['KaiTi'] # 指定默认字体

x = range(len(df))
plt.figure(figsize=(7,5))
plt.plot(x,df['PRICE'])
plt.plot(x,df['RM'])
plt.title("变量相关性曲线")
plt.xlabel("X轴")
plt.ylabel("Y轴")
plt.legend()

③看看各个字段与价格的散点图：以初步了解价格与相应字段的关系。

In [ ]:
x_data = df[['CRIM','ZN','INDUS','CHAS','NOX','RM','GE','DIS','RAD','TAX','PRTATIO','B','LSTAT']] # 导入所有特征变量
y_data = df[['PRICE']] # 导入回归目标值（房价）

plt.figure(figsize=(18,18))

for i in range(13):
    plt.subplot(4,4,i+1) #i+1 当前绘制的是哪个子图
    plt.scatter(x_data.values[:,i],y_data,s = 5)    #.values将DataFrame对象X_df转成ndarray数组
    plt.xlabel(names[i])
    plt.ylabel('Price')
    plt.title(str(i+1)+'. '+names[i]+' - Price')  
    
plt.tight_layout()
plt.show()

可以看到不是所有的字段与价格都有较强的相关关系，但本例中不涉及多元线性回归的向后删除，仅做最简单的多元性性回归的分析处理。

可以看到房子价格跟住宅的房间数成比较强的正相关，而跟低收入人数比例有比较强的负相关。

In [ ]:
# 对每列数据做归一化
scaler = MinMaxScaler()
data_T_minmax=scaler.fit_transform(df.iloc[:,:-1].values.T)
data_minmax = data_T_minmax.T
print(data_minmax)

In [ ]:

y_data = df[['PRICE']] # 导入目标值（房价）

plt.figure(figsize=(18,18))

for i in range(13):
    plt.subplot(4,4,i+1)
    plt.scatter(data_minmax[:,i],y_data,s = 5)    #.values将DataFrame对象X_df转成ndarray数组
    plt.xlabel(names[i])
    plt.ylabel('Price')
    plt.title(str(i+1)+'. '+names[i]+' - Price')  
    
plt.tight_layout()
plt.show()

In [ ]:
#相关系数计算

corrboston = df.corr()
print(corrboston)

plt.figure(figsize=(10,10))    #设置画布
sns.heatmap(corrboston,annot=True,cmap='RdGy')
plt.show()

In [10]:
# 单独计算连个变量的相关系数
print(df['PRICE'].corr(df['RM']))

NameError: name 'df' is not defined

In [ ]:
# 协方差计算
covboston = df.cov()
print(covboston)

In [ ]:
# 单独计算两个变量的协方差
print(df['PRICE'].cov(df['RM']))

In [ ]:
#  seaborn画的散点图 选择三列特征
cols = ['RM', 'PRICE', 'LSTAT']
# 构造三列特征之间的联系即构造散点图矩阵
sns.pairplot(df[cols], height=3)
plt.tight_layout()
plt.show()

三、预测性数据分析

①选取线性回归字段：

In [13]:
from sklearn import linear_model

#定义线性回归的x和y变量
x=pd.DataFrame(df[['CRIM','ZN','INDUS','CHAS','NOX','RM','GE','DIS','RAD','TAX','PRTATIO','B','LSTAT']])
y=df['PRICE']
x

D:\anaconda\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
D:\anaconda\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).ep

NameError: name 'df' is not defined

②建立线性回归模型，并调用：可以看到各个字段的回归系数，可以写出一个回归方程：y=ax1+bx2+……，理论上你知道一套新房子的各个字段，带入公式即可预测出价格。

In [12]:
#建立线性回归模型，并将变量带入模型进行训练。
clf = linear_model.LinearRegression()
clf.fit(x, y)

#查看回归系数。本例为一元回归,所以只有一个系数。
print('回归系数:', clf.coef_)

NameError: name 'linear_model' is not defined

③可以进行简单的预测：

In [ ]:
y_pred =clf.predict(x)
# print(y_pred)

④计算回归系数：计算出模型的拟合优度为0.74，回归拟合效果较好。

In [ ]:
# 导入评估标准 r2_score
from sklearn.metrics import r2_score
score = r2_score(y, y_pred)
score

⑤模型可视化

In [11]:
#用RM预测PRICE
plt.rcParams['font.sans-serif'] = ['KaiTi'] # 指定默认字体

clf = linear_model.LinearRegression()
x = df[['RM']].values
clf.fit(x, y)

plt.scatter(x, y, c='r', s=30, edgecolor='white',label='训练数据')
plt.plot(x, clf.predict(x), c='g')
plt.xlabel('平均房间数目[PRICE]')
plt.ylabel('以1000美元为计价单位的房价[RM]')
plt.title('波士顿房价预测', fontsize=20)
plt.legend()
plt.show()
print('普通线性回归斜率:{}'.format(clf.coef_[0]))


NameError: name 'linear_model' is not defined